<div style="background-color:#000;"><img src="pqn.png"></img></div>

This notebook demonstrates the use of VectorBT PRO for backtesting and analyzing quantitative trading strategies focusing on chart patterns. It provides tools for data acquisition, signal generation, portfolio optimization, and strategy simulation. The notebook includes a pattern detector optimized with Numba to scan data for specified patterns. We use hourly price data from TradingView to detect and backtest various bullish and bearish patterns. Practical applications include testing trading strategies, understanding historical market behavior, and optimizing algorithmic trading setups.

In [ ]:
import vectorbtpro as vbt
import pandas as pd
import numpy as np

In [ ]:
vbt.settings.set_theme("dark")

Define the symbols for which we will pull data from TradingView.

In [ ]:
symbols = [
    "NASDAQ:META",
    "NASDAQ:AMZN",
    "NASDAQ:AAPL",
    "NASDAQ:NFLX",
    "NASDAQ:GOOG",
]

Pull hourly price data for the defined symbols from TradingView.

In [ ]:
data = vbt.TVData.pull(symbols, timeframe="hourly")

Define the date range for the analysis.

In [ ]:
start_date = "2020"
end_date = None

Slice the data to include only the specified date range.

In [ ]:
data = data.xloc[start_date:end_date]

Display statistics of the data to ensure it spans the correct date period and is free of NaN values.

In [ ]:
print(data.stats())

Calculate the HLC3 (High-Low-Close average) as the chosen feature for pattern detection.

In [ ]:
price = data.hlc3

Define bullish and bearish chart patterns as numerical sequences.

In [ ]:
bullish_patterns = {
    "double_bottom": [5, 1, 3, 1, 5],
    "exp_triangle": [3, 4, 2, 5, 1, 6],
    "asc_triangle": [1, 5, 2, 5, 3, 6],
    "symm_triangle": [1, 6, 2, 5, 3, 6],
    "pennant": [6, 1, 5, 2, 4, 3, 6]
}
bearish_patterns = {
    "head_and_shoulders": [1, 4, 2, 6, 2, 4, 1],
    "double_top": [1, 5, 3, 5, 1],
    "desc_triangle": [6, 2, 5, 2, 4, 1],
    "symm_triangle": [6, 1, 5, 2, 4, 1],
    "pennant": [1, 6, 2, 5, 3, 4, 1]
}

Plot the numerical sequence of the "double_bottom" pattern for visual confirmation.

In [ ]:
pd.Series(bullish_patterns["double_bottom"]).vbt.plot().show_png()

Define the minimum and maximum window lengths for pattern detection.

In [ ]:
min_window = 24
max_window = 24 * 30

Define a function to detect patterns in the price data.

In [ ]:
def detect_patterns(patterns):
    """Detect patterns in price data.

    This function searches for specified patterns in the price data within defined windows.

    Parameters
    ----------
    patterns : dict
        Dictionary of patterns to detect.

    Returns
    -------
    vbt.PatternRanges
        Detected pattern ranges.
    """
    
    return vbt.PatternRanges.from_pattern_search(
        price,
        open=data.open,
        high=data.high,
        low=data.low,
        close=data.close,
        pattern=patterns,
        window=min_window,
        max_window=max_window,
        execute_kwargs=dict(
            engine="threadpool",
            chunk_len="auto",
            show_progress=True
        )
    )

Detect bullish and bearish patterns in the price data.

In [ ]:
bullish_matches = detect_patterns(
    vbt.Param(
        bullish_patterns,
        name="bullish_pattern"
    )
)
bearish_matches = detect_patterns(
    vbt.Param(
        bearish_patterns,
        name="bearish_pattern"
    )
)

Print the number of matches for each bullish pattern and dataset.

In [ ]:
print(bullish_matches.count())

Plot the pattern and dataset with the most matches.

In [ ]:
vbt.settings.plotting.auto_rangebreaks = True

In [ ]:
display_column = bullish_matches.count().idxmax()

In [ ]:
bullish_matches.plot(column=display_column, fit_ranges=True).show_png()

Zoom in on a specific match within the dataset.

In [ ]:
display_match = 3

In [ ]:
bullish_matches.plot(
    column=display_column,
    fit_ranges=display_match
).show_png()

Convert detected patterns into entry and exit signals for backtesting.

In [ ]:
entries = bullish_matches.last_pd_mask
exits = bearish_matches.last_pd_mask

Generate a Cartesian product of bullish and bearish patterns for systematic testing.

In [ ]:
entries, exits = entries.vbt.x(exits)

Print the columns representing individual backtests.

In [ ]:
print(entries.columns)

Simulate a portfolio using the generated signals.

In [ ]:
pf = vbt.Portfolio.from_signals(data, entries, exits)

Calculate and print the mean total return for each combination of bullish and bearish patterns.

In [ ]:
mean_total_return = pf.total_return.groupby(
    ["bullish_pattern", "bearish_pattern"]
).mean()

In [ ]:
print(mean_total_return)

Visualize the mean total return as a heatmap.

In [ ]:
mean_total_return.vbt.heatmap(
    x_level="bearish_pattern",
    y_level="bullish_pattern"
).show_png()

<a href="https://pyquantnews.com/">PyQuant News</a> is where finance practitioners level up with Python for quant finance, algorithmic trading, and market data analysis. Looking to get started? Check out the fastest growing, top-selling course to <a href="https://gettingstartedwithpythonforquantfinance.com/">get started with Python for quant finance</a>. For educational purposes. Not investment advise. Use at your own risk.